### Clone keras yolov3 repository

In [1]:
%cd /data1
!git clone https://github.com/qqwweee/keras-yolo3.git keras_yolo3

/data1
Cloning into 'keras_yolo3'...
remote: Enumerating objects: 144, done.
remote: Total 144 (delta 0), reused 0 (delta 0), pack-reused 144
Receiving objects: 100% (144/144), 151.08 KiB | 762.00 KiB/s, done.
Resolving deltas: 100% (65/65), done.


### Load and convert yolov3 model to keras

In [2]:
%cd keras_yolo3
!wget https://pjreddie.com/media/files/yolov3.weights
!python convert.py yolov3.cfg yolov3.weights /data1/yolo.h5

/data1/keras_yolo3
--2021-08-24 08:24:40--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  8.90MB/s    in 30s     

2021-08-24 08:25:11 (8.00 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]

/workspace/miniconda/lib/python3.7/site-packages/numpy/__init__.py:156: UserWarning: mkl-service package failed to import, therefore Intel(R) MKL initialization ensuring its correct out-of-the box operation under condition when Gnu OpenMP had already been loaded by Python process is not assured. Please install mkl-service package, see http://github.com/IntelPython/mkl-service
  from . import _distributor_init
Using TensorFlow backend.
Loading weights.
Weights Header:  0 2 0 [32013312]

### Download kneron images for further quantization

In [3]:
%cd /data1
!wget http://doc.kneron.com/docs/toolchain/res/test_image10.zip
!unzip test_image10.zip

/data1
--2021-08-24 08:26:26--  http://doc.kneron.com/docs/toolchain/res/test_image10.zip
Resolving doc.kneron.com (doc.kneron.com)... 75.26.5.105
Connecting to doc.kneron.com (doc.kneron.com)|75.26.5.105|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2615052 (2.5M) [application/zip]
Saving to: ‘test_image10.zip’

test_image10.zip    100%[===================>]   2.49M  1.49MB/s    in 1.7s    

2021-08-24 08:26:28 (1.49 MB/s) - ‘test_image10.zip’ saved [2615052/2615052]

Archive:  test_image10.zip
   creating: test_image10/
 extracting: test_image10/000000000139.jpg  
  inflating: test_image10/000000000785.jpg  
  inflating: test_image10/000000000872.jpg  
  inflating: test_image10/000000000885.jpg  
 extracting: test_image10/000000001000.jpg  
  inflating: test_image10/000000001268.jpg  
  inflating: test_image10/000000001296.jpg  
  inflating: test_image10/000000005001.jpg  
  inflating: test_image10/000000005193.jpg  
  inflating: test_image10/309_190.jpg  


### Copy image for model's testing in different convertation stages

In [4]:
%cd /data1
!cp /workspace/E2E_Simulator/app/test_image_folder/yolo/000000350003.jpg ./.

/data1


### Run example script test.py to convert yolov3 from keras to nef

In [ ]:
!python /workspace/Kneron_yolov3_inference/test.py

### Make changes to E2E simulator in order to use it with multiprocessing

In [6]:
!cp /workspace/Kneron_yolov3_inference/model.py  /data1/keras_yolo3/yolo3/model.py 
!cp /workspace/Kneron_yolov3_inference/nef.py /workspace/E2E_Simulator/python_flow/nef/nef.py  
!cp /workspace/Kneron_yolov3_inference/kneron_inference.py /workspace/E2E_Simulator/python_flow/kneron_inference.py 

## Download COCO dataset

In [ ]:
!mkdir /workspace/COCO 
%cd /workspace/COCO

In [ ]:
!wget -c http://images.cocodataset.org/zips/val2014.zip
!unzip -q val2014.zip
!rm val2014.zip

In [ ]:
!mv val2014 images 
!mkdir val2014
!mv images val2014/images
!mkdir /workspace/COCO/val2014/predictions

## download labels

In [ ]:
%cd val2014
!wget https://pjreddie.com/media/files/coco/5k.part

In [ ]:
!wget -c https://pjreddie.com/media/files/coco/labels.tgz
!tar xzf labels.tgz
!rm labels.tgz
!mv labels/val2014 val2014
import shutil
shutil.rmtree('labels')
!mv val2014 labels

In [ ]:
val = open('5k.part')
val = val.read()
val= val.split("\n")
validation = []
for image in val:
    image = image.split("/")
    validation.append(image[-1][:-4])

## delete 35k redundant images from val2014

In [ ]:
import os
from time import time
from os import listdir
from os.path import isfile, join
path = '/workspace/COCO/val2014' # path to validation directory
predictions = path + '/' + 'predictions'
names = path + '/' +  'images'
predictions =[f[:-4] for f in listdir(predictions) if isfile(join(predictions, f))] 
names =[f[:-4] for f in listdir(names) if isfile(join(names, f))]

predicted = list(set(validation) & set(names))
redundant = names

for element in predicted:
    redundant.remove(element) 



In [ ]:
for name in redundant:
    os.remove("/workspace/COCO/val2014/images/" + name + ".jpg")
    try:
        os.remove("/workspace/COCO/val2014/labels/" + name + ".txt")
    except:
        print("Empty label")

In [109]:
names = path + '/' +  'images'
names =[f[:-4] for f in listdir(names) if isfile(join(names, f))]
labels = path + '/' +  'labels' 
labels =[f[:-4] for f in listdir(labels) if isfile(join(labels, f))]
not_existing = names
for label in labels:
    #print(label)
    not_existing.remove(label)

In [ ]:
for label in not_existing:
    with open('/workspace/COCO/val2014/labels/' + label + '.txt', 'w') as f:
        f.write("")
        print("Writing empty label")

# Yolov3 

In [10]:
%cd /workspace/Kneron_yolov3_inference/

/workspace/Kneron_yolov3_inference


In [11]:
!python parallel.py -h

/workspace/miniconda/lib/python3.7/site-packages/numpy/__init__.py:156: UserWarning: mkl-service package failed to import, therefore Intel(R) MKL initialization ensuring its correct out-of-the box operation under condition when Gnu OpenMP had already been loaded by Python process is not assured. Please install mkl-service package, see http://github.com/IntelPython/mkl-service
  from . import _distributor_init
Using TensorFlow backend.
usage: parallel.py [-h] [--demo] [--image IMAGE] [--path PATH] [--nef NEF]
                   [--step STEP] [--init INIT] [--model MODEL]
                   [--threads THREADS] [--img-size IMG_SIZE] [--conf-t CONF_T]
                   [--iou-t IOU_T]

Runs an inference on multiple images.

optional arguments:
  -h, --help           show this help message and exit
  --demo               run demo on your image
  --image IMAGE        path to your image for demo
  --path PATH          directory of your images
  --nef NEF            path to your nef model
  -

In [ ]:
"""
This script will run untill no more predictions to be done.
We recommend to increase number of models, if you're using it to simulate Kneron 720, 
since this simulator uses only one thread.
"""

import multiprocessing
import os
from time import time
from os import listdir
from os.path import isfile, join
tic = time() 
step = 5 # images per iteration of script
models = 5 # number of models running in parallel
init = 3 # time between model's initialization
threads = 16 # number of workers for each model, works only for 520 model
path = '/workspace/COCO/val2014' # path to validation directory
predictions = path + '/' + 'predictions'
names = path + '/' +  'images'


def missed(predictions, names):    
    predictions =[f[:-4] for f in listdir(predictions) if isfile(join(predictions, f))]    
    names =[f[:-4] for f in listdir(names) if isfile(join(names, f))]
    predicted = list(set(predictions) & set(names))
    not_predicted = names
    
    for element in predicted:
        #print(element)
        not_predicted.remove(element) 

    return not_predicted

# Check if there is all predictions
not_predicted = missed(predictions, names)
all_processes =[]
for i in range(models):
    all_processes.append('parallel.py --path {} --model {} --step {} --init {} --conf-t 0.001 --threads {}'.format(path, i, step, init, threads))

while len(not_predicted):
    # This block of code enables us to call the script from command line.                                                                                
    def execute(process):                                                             
        os.system(f'python {process}')                                       


    process_pool = multiprocessing.Pool(processes = models)                                                        
    process_pool.map(execute, all_processes)
    
    # Check if there is all predictions
    not_predicted = missed(predictions, names) 
    
print("Script ran for ", time() - tic)